![First Power Bi](.\images\dbachecks-logo.png )
# Creating the Power Bi

Displaying the results in Power Bi is another capability of dbachecks. You can use this to build dashboards for your team or enable quick identification of issues that need to be resolved. There is a Power Bi dashboard embedded with the dbachecks module that you downloaded. This will automatically use the json file that is created when you use the command `Update-DbcPowerBiDataSource`

You **MUST** use the `-PassThru` parameter of `Invoke-DbcCheck` when running the checks otherwise `Update-PowerBiDataSource` will not have an object to work with. There is no way to check that you are going to use `Update-DbcPowerBiDataSource` so the first time you will realise that youhave not used `-Passthru` is after the checks have run which can be very frustrating when it has taken an hour or two to create the configuration and run the checks as the beard has found out!

With your configuration set up you run as you wish, you run `Invoke-DbcCheck` with the `-PassThru` parameter. 

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
$null = Reset-DbcConfig
$invokeDbcCheckSplat = @{
    Check = 'DatabaseExists', 'DatabaseStatus', 'DefaultFilePath', 'XpCmdShellDisabled', 'VirtualLogFile', 'TempDbConfiguration', 'MaxMemory', 'ModelDbGrowth', 'LogfileCount', 'LastGoodCheckDb'
    SqlInstance = $SqlInstances
    SqlCredential = $SqlCredential
    Show = 'Summary'
}

Invoke-DbcCheck @invokeDbcCheckSplat -PassThru | Update-DbcPowerBiDataSource

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Database.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Instance.Tests.ps1' with Tags DatabaseExists', 'DatabaseStatus', 'DefaultFilePath', 'XpCmdShellDisabled', 'VirtualLogFile', 'TempDbConfiguration', 'MaxMemory', 'ModelDbGrowth', 'LogfileCount', 'LastGoodCheckDb
Tests completed in 15.59s
Tests Passed: 84, Failed: 8, Skipped: 2, Pending: 0, Inconclusive: 0 
[14:11:06][Update-DbcPowerBiDataSource] Wrote results to C:\Windows\temp\dbachecks\dbachecks_1_Default__DatabaseExists_DatabaseStatus_DefaultFilePath_XpCmdShellDisabled.json


You will also notice that for the Show Parameter, we have used `Summary` this will only display the summary and not any of the check results.

The output will also show that the file that has been created is located in the windows temp directory. This is where the includd Power Bi expects to find the files. You can open the Power Bi file using `Start-DbcPowerBi`

In [2]:
Start-DbcPowerBi

[14:11:41][Start-DbcPowerBi] Launching the dbachecks dashboard. This may take a moment.


This will look ***exactly*** like this.

![First Power Bi](.\images\firstpowerbi.png )

This is the default view and to view your results you will need to refresh the data.

![Refresh Power Bi](.\images\RefreshPowerBi.gif )



# Combining Results

Now the power of dbachecks is really released. You can create configurations for different types of checks, load the configuration, run the checks and then save ready for the Power Bi. Using the `-Environment` parameter of `Update-DbcPowerBiDataSource` you can enable filtering by the different sets of checks.

The code below will use the configuration files created in Notebook 04 so you will need to have run through that notebook prior to running this code.

In [3]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Morning-Checks-Production.json 
$invokeDbcCheckSplat = @{
    SqlCredential = $SqlCredential
    Check = 'InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth'
    Show = 'Summary'
}
Invoke-DbcCheck @invokeDbcCheckSplat -PassThru | Update-DbcPowerBiDataSource -Environment Morning-Checks-Production


$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\New-Instance.json 
$invokeDbcCheckSplat = @{
    SqlInstance = 'localhost,15592'
    SqlCredential = $SqlCredential
    Check = 'Instance','Agent'
    Show = 'Summary'
}
Invoke-DbcCheck @invokeDbcCheckSplat -PassThru | Update-DbcPowerBiDataSource -Environment New-Instance

$null = Reset-DbcConfig
$null = Import-DbcConfig -Path $FolderPath\Weekly-Check.json 
$invokeDbcCheckSplat = @{
    SqlInstance = 'localhost,15592'
    SqlCredential = $SqlCredential
    Check = 'Agent', 'Database','Instance'
    Show = 'Summary'
}
Invoke-DbcCheck @invokeDbcCheckSplat -PassThru | Update-DbcPowerBiDataSource -Environment Weekly-Check

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Agent.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Database.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Instance.Tests.ps1', 'C:\Users\mrrob\Documents\PowerShell\Modules\dbachecks\1.2.25\checks\Server.Tests.ps1' with Tags InstanceConnection', 'FailedJob', 'LastBackup', 'ErrorLog', 'DiskCapacity', 'MemoryDump', 'LastJobRunTime', 'LongRunningJob', 'LogfileSize', 'FutureFileGrowth
Tests completed in 73.71s
Tests Passed: 69, Failed: 20, Skipped: 2, Pending: 0, Inconclusive: 0 
[14:14:30][Update-DbcPowerBiDataSource] Wrote results to C:\Windows\temp\dbachecks\dbachecks_1_Morning-Checks-Production__InstanceConnection_FailedJob_LastBackup_ErrorLog.json
    ____         

Tests completed in 53.85s
Tests Passed: 819, Failed: 51, Skipped: 24, Pending: 0, Inconclusive: 0 
[14:15:58][Update-DbcPowerBiDataSource] Wrote results to C:\Windows\temp\dbachecks\dbachecks_1_Weekly-Check__Agent_Database_Instance.json


Now you can open the PowerBi with `Start-DbcPowerBi` and start exploring the results (and fixing the broken things!) 

[![PowerBi](./images/video.png)](https://www.youtube.com/watch?v=W7LRl2zKbzs)

Clicking the image will open the video in YouTube

If you have Power Bi open you can just refresh the data source to see the new data

In [1]:
Start-DbcPowerBi

[09:12:12][Start-DbcPowerBi] Launching the dbachecks dashboard. This may take a moment.


In [2]:
Start-DbcPowerBi -FromDatabase

[

14:13:01

][

Start-DbcPowerBi

] Launching the dbachecks dashboard. This may take a moment.
